In [14]:
import os    
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import dataset
import pandas as pd
import utlis

In [15]:
def ungroupby(df):
    df["bboxes"]=df.apply(lambda x: utlis.create_boxes(x["image"].shape[:2],x["boxes"]),axis=1)
    s=df['bboxes'].apply(lambda x: pd.Series(pd.Series(x)), 1).stack()
    s.index = s.index.droplevel(-1)
    bboxes_df=s.apply(pd.Series)
    bboxes_df=bboxes_df.join(df["ImgName"])  
    bboxes_df=bboxes_df.reset_index()
    bboxes_df.columns=["ImgId","label_index", "xcenter", "ycenter", "w", "h","ImgName"]
    bboxes_df["label_index"]=bboxes_df["label_index"].apply(int)
    return bboxes_df

In [16]:
df_train=dataset.create_df("train",with_image_names=True)


In [17]:
train_bboxs=ungroupby(df_train)


In [18]:
train_bboxs

,ImgId,label_index,xcenter,ycenter,w,h,ImgName
0,0,2,0.445657,0.404520,0.225424,0.219209,P040_balloon1_2500
1,0,7,0.627013,0.525989,0.210169,0.227119,P040_balloon1_2500
2,1,5,0.580750,0.527989,0.120718,0.241304,P019_balloon1_3236
3,1,2,0.398450,0.532880,0.230832,0.290217,P019_balloon1_3236
4,2,7,0.647511,0.522177,0.138588,0.225606,P030_balloon1_6865
...,...,...,...,...,...,...,...
1569,787,2,0.462072,0.497011,0.235726,0.214130,P025_balloon1_3977
1570,788,5,0.671121,0.543193,0.126206,0.185149,P027_balloon1_4255
1571,788,2,0.457684,0.532302,0.259094,0.238614,P027_balloon1_4255
1572,789,5,0.631321,0.516576,0.159869,0.253261,P017_balloon1_1862


In [6]:
alpha = 1.5 # Contrast control
beta = 10 # Brightness control

In [22]:
ids = train_bboxs['ImgId'].unique()
for i in ids:
    temp_df = train_bboxs.loc[train_bboxs['ImgId'] == i]
    #print(temp_df.iloc[0]['ImgName']) 
    name_orig = temp_df.iloc[0]['ImgName']#.split("_")[1:]
    #name_of_file = "_".join(name_orig)
    #print(name_orig)
    image = cv2.imread("images/train/"+name_orig+".jpg")
    adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
    #plt.imshow(adjusted)
    cv2.imwrite("images_adjusted/train/"+"adjusted_"+temp_df.iloc[0]['ImgName']+'.jpg',adjusted)
    #labels_data = combine_data(temp_df)
    if temp_df.shape[0] == 1:
        data = temp_df.iloc[0][['label_index','xcenter','ycenter','w','h']]
        data_str = [str(x) for x in data]
        data_final_str = ' '.join(data_str)
        lines =  [data_final_str]
    else:
        data = list(temp_df.iloc[0][['label_index','xcenter','ycenter','w','h']])
        data2 = list(temp_df.iloc[1][['label_index','xcenter','ycenter','w','h']])
        data_str = [str(x) for x in data]
        data_final_str = ' '.join(data_str)
        data_str2 = [str(x) for x in data2]
        data_final_str2 = ' '.join(data_str2)
        lines = [data_final_str,data_final_str2]
    with open('labels_adjusted/train/'+"adjusted_"+temp_df.iloc[0]['ImgName']+'.txt', 'w') as f:
        for line in lines:
            f.write(f"{line}\n")
    
    

    